In [1]:
import os,cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD,RMSprop,adam
import tensorflow as tf

Using TensorFlow backend.


In [2]:
os.chdir('/home/student/Documents/PCB_3/Train_Data')
train_x = np.load('train_X.npy')
train_x = train_x[:,100:200,100:200,:];
train_y = np.load('train_y.npy')

In [3]:
num_classes = 2
print(np.unique(train_y,return_counts=True))
train_y= np_utils.to_categorical(train_y, num_classes)

(array([0., 1.]), array([15000, 15000]))


In [4]:
print(len(train_x))
print(len(train_y))
print(train_x.shape)
print(train_y.shape)

30000
30000
(30000, 100, 100, 3)
(30000, 2)


In [5]:
mean = np.mean(train_x)
std = np.std(train_x)
print (mean, std)

train_x = train_x - mean
train_x = train_x / std
print(train_x.shape)
print(train_y.shape)

62.34926454777778 58.207460392701165
(30000, 100, 100, 3)
(30000, 2)


In [6]:
X_train, X_valid, y_train, y_valid = train_test_split(train_x, train_y, test_size=0.2, random_state=2)

print(len(X_train))
print(len(y_train))

print("")

print(len(X_valid))
print(len(y_valid))

print("")

print(X_train.shape)
print(y_train.shape)

print("")

print(X_valid.shape)
print(y_valid.shape)

24000
24000

6000
6000

(24000, 100, 100, 3)
(24000, 2)

(6000, 100, 100, 3)
(6000, 2)


In [7]:
X_train = X_train.astype('float32')
y_train = y_train.astype('float32')
#y_train = np.reshape(y_train,(-1,1))

X_valid = X_valid.astype('float32')
y_valid = y_valid.astype('float32')
#y_valid = np.reshape(y_valid,(-1,1))


print(X_train.shape)
print(y_train.shape)
print("")
print(X_valid.shape)
print(y_valid.shape)

(24000, 100, 100, 3)
(24000, 2)

(6000, 100, 100, 3)
(6000, 2)


In [8]:
from keras import layers
from keras.models import Model

def _after_conv(in_tensor):
    norm = layers.BatchNormalization()(in_tensor)
    return layers.Activation('relu')(norm)

def conv1(in_tensor, filters):
    conv = layers.Conv2D(filters, kernel_size=1, strides=1)(in_tensor)
    return _after_conv(conv)

def conv1_downsample(in_tensor, filters):
    conv = layers.Conv2D(filters, kernel_size=1, strides=2)(in_tensor)
    return _after_conv(conv)

def conv3(in_tensor, filters):
    conv = layers.Conv2D(filters, kernel_size=3, strides=1, padding='same')(in_tensor)
    return _after_conv(conv)

def conv3_downsample(in_tensor, filters):
    conv = layers.Conv2D(filters, kernel_size=3, strides=2, padding='same')(in_tensor)
    return _after_conv(conv)

def resnet_block_wo_bottlneck(in_tensor, filters, downsample=False):
    if downsample:
        conv1_rb = conv3_downsample(in_tensor, filters)
    else:
        conv1_rb = conv3(in_tensor, filters)
    conv2_rb = conv3(conv1_rb, filters)

    if downsample:
        in_tensor = conv1_downsample(in_tensor, filters)
    result = layers.Add()([conv2_rb, in_tensor])

    return layers.Activation('relu')(result)

def resnet_block_w_bottlneck(in_tensor,
                             filters,
                             downsample=False,
                             change_channels=False):
    if downsample:
        conv1_rb = conv1_downsample(in_tensor, int(filters/4))
    else:
        conv1_rb = conv1(in_tensor, int(filters/4))
    conv2_rb = conv3(conv1_rb, int(filters/4))
    conv3_rb = conv1(conv2_rb, filters)

    if downsample:
        in_tensor = conv1_downsample(in_tensor, filters)
    elif change_channels:
        in_tensor = conv1(in_tensor, filters)
    result = layers.Add()([conv3_rb, in_tensor])

    return result

def _pre_res_blocks(in_tensor):
    conv = layers.Conv2D(64, 7, strides=2, padding='same')(in_tensor)
    conv = _after_conv(conv)
    pool = layers.MaxPool2D(3, 2, padding='same')(conv)
    return pool

def _post_res_blocks(in_tensor, n_classes):
    pool = layers.GlobalAvgPool2D()(in_tensor)
    preds = layers.Dense(n_classes, activation='softmax')(pool)
    return preds

def convx_wo_bottleneck(in_tensor, filters, n_times, downsample_1=False):
    res = in_tensor
    for i in range(n_times):
        if i == 0:
            res = resnet_block_wo_bottlneck(res, filters, downsample_1)
        else:
            res = resnet_block_wo_bottlneck(res, filters)
    return res

def convx_w_bottleneck(in_tensor, filters, n_times, downsample_1=False):
    res = in_tensor
    for i in range(n_times):
        if i == 0:
            res = resnet_block_w_bottlneck(res, filters, downsample_1, not downsample_1)
        else:
            res = resnet_block_w_bottlneck(res, filters)
    return res

def _resnet(in_shape=(224,224,3),
            n_classes=1000,
            convx=[64, 128, 256, 512],
            n_convx=[2, 2, 2, 2],
            convx_fn=convx_wo_bottleneck):
    in_layer = layers.Input(in_shape)

    downsampled = _pre_res_blocks(in_layer)

    conv2x = convx_fn(downsampled, convx[0], n_convx[0])
    conv3x = convx_fn(conv2x, convx[1], n_convx[1], True)
    conv4x = convx_fn(conv3x, convx[2], n_convx[2], True)
    conv5x = convx_fn(conv4x, convx[3], n_convx[3], True)

    preds = _post_res_blocks(conv5x, n_classes)

    model = Model(in_layer, preds)
    return model

def resnet18(in_shape=(100,100,3), n_classes=2):
    return _resnet(in_shape, n_classes)

def resnet34(in_shape=(300,300,3), n_classes=2):
    return _resnet(in_shape,n_classes,n_convx=[3, 4, 6, 3])

def resnet50(in_shape=(300,300,3), n_classes=2):
    return _resnet(in_shape,n_classes,[256, 512, 1024, 2048],[3, 4, 6, 3],convx_w_bottleneck)

def resnet101(in_shape=(300,300,3), n_classes=2):
    return _resnet(in_shape,n_classes,[256, 512, 1024, 2048],[3, 4, 23, 3],convx_w_bottleneck)

def resnet152(in_shape=(300,300,3), n_classes=2):
    return _resnet(in_shape,n_classes,[256, 512, 1024, 2048],[3, 8, 36, 3],convx_w_bottleneck)

if __name__ == '__main__':
    model = resnet18()
    print(model.summary())

W1209 10:14:54.871383 140077872379712 deprecation_wrapper.py:119] From /home/student/anaconda3/envs/gpu/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1209 10:14:54.887747 140077872379712 deprecation_wrapper.py:119] From /home/student/anaconda3/envs/gpu/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1209 10:14:54.891199 140077872379712 deprecation_wrapper.py:119] From /home/student/anaconda3/envs/gpu/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1209 10:14:54.914226 140077872379712 deprecation_wrapper.py:119] From /home/student/anaconda3/envs/gpu/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100, 100, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 50, 50, 64)   9472        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 50, 50, 64)   256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 50, 50, 64)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
max_poolin

In [9]:
os.chdir('/home/student/Documents/PCB_3/Models/Res18/Results')
from keras.callbacks import ModelCheckpoint,TensorBoard,CSVLogger,ReduceLROnPlateau,LearningRateScheduler,EarlyStopping
mc = ModelCheckpoint('PCB3_Res18_Aug_data_100*100_2.h5', monitor='val_loss', mode='min',save_best_only=True)
es = EarlyStopping(monitor='val_loss', min_delta=0, patience=25, verbose=0, mode='auto', baseline=None, restore_best_weights=True)
cv = CSVLogger('PCB3_Res18_Aug_data_100*100_2.csv',append=True)
tensorboard_callback = TensorBoard(log_dir='logs', histogram_freq=1)

In [10]:
import keras
adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

W1209 10:15:16.249745 140077872379712 deprecation_wrapper.py:119] From /home/student/anaconda3/envs/gpu/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [11]:
batch_size = 32
epochs = 100
history = model.fit(X_train,y_train,batch_size=batch_size,epochs=epochs,validation_data=(X_valid,y_valid),verbose = 1,callbacks=[mc,cv,tensorboard_callback],shuffle=True)    

W1209 10:15:21.346643 140077872379712 deprecation.py:323] From /home/student/anaconda3/envs/gpu/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 24000 samples, validate on 6000 samples


W1209 10:15:25.761233 140077872379712 deprecation_wrapper.py:119] From /home/student/anaconda3/envs/gpu/lib/python3.6/site-packages/keras/callbacks.py:848: The name tf.summary.histogram is deprecated. Please use tf.compat.v1.summary.histogram instead.

W1209 10:15:25.948509 140077872379712 deprecation_wrapper.py:119] From /home/student/anaconda3/envs/gpu/lib/python3.6/site-packages/keras/callbacks.py:850: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W1209 10:15:25.955152 140077872379712 deprecation_wrapper.py:119] From /home/student/anaconda3/envs/gpu/lib/python3.6/site-packages/keras/callbacks.py:853: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



Epoch 1/100
24000/24000 [==============================] - 46s 2ms/step - loss: 0.3870 - acc: 0.8408 - val_loss: 0.2585 - val_acc: 0.9010
Epoch 2/100
24000/24000 [==============================] - 42s 2ms/step - loss: 0.2357 - acc: 0.9103 - val_loss: 0.7931 - val_acc: 0.7113
Epoch 3/100
24000/24000 [==============================] - 43s 2ms/step - loss: 0.1955 - acc: 0.9263 - val_loss: 0.1967 - val_acc: 0.9248
Epoch 4/100
24000/24000 [==============================] - 42s 2ms/step - loss: 0.1738 - acc: 0.9343 - val_loss: 0.1968 - val_acc: 0.9253
Epoch 5/100
24000/24000 [==============================] - 42s 2ms/step - loss: 0.1598 - acc: 0.9405 - val_loss: 0.2893 - val_acc: 0.9138
Epoch 6/100
24000/24000 [==============================] - 43s 2ms/step - loss: 0.1436 - acc: 0.9476 - val_loss: 0.1907 - val_acc: 0.9278
Epoch 7/100
24000/24000 [==============================] - 43s 2ms/step - loss: 0.1265 - acc: 0.9517 - val_loss: 3.7258 - val_acc: 0.7017
Epoch 8/100
24000/24000 [=========

24000/24000 [==============================] - 41s 2ms/step - loss: 0.0075 - acc: 0.9973 - val_loss: 0.2569 - val_acc: 0.9545
Epoch 61/100
24000/24000 [==============================] - 41s 2ms/step - loss: 0.0105 - acc: 0.9964 - val_loss: 0.1869 - val_acc: 0.9607
Epoch 62/100
24000/24000 [==============================] - 41s 2ms/step - loss: 0.0072 - acc: 0.9977 - val_loss: 0.2767 - val_acc: 0.9405
Epoch 63/100
24000/24000 [==============================] - 41s 2ms/step - loss: 0.0067 - acc: 0.9978 - val_loss: 0.2078 - val_acc: 0.9580
Epoch 64/100
24000/24000 [==============================] - 41s 2ms/step - loss: 0.0073 - acc: 0.9975 - val_loss: 0.2299 - val_acc: 0.9585
Epoch 65/100
24000/24000 [==============================] - 41s 2ms/step - loss: 0.0054 - acc: 0.9981 - val_loss: 0.2284 - val_acc: 0.9580
Epoch 66/100
24000/24000 [==============================] - 41s 2ms/step - loss: 0.0111 - acc: 0.9969 - val_loss: 0.2054 - val_acc: 0.9563
Epoch 67/100
24000/24000 [==============